In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
import csv
import time
from scipy import ndimage
from tensorflow.keras import callbacks
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

TRAIN_DIR = 'createdData/'
IMG_SIZE = 300
LR = 0.001
training_data =[]
labelz = dict(enumerate(['1', '2','3','4','5','6','7','8','9','10','11','12','13','14']))


Using TensorFlow backend.


In [0]:
# training_data = np.load('training_data.npy', allow_pickle=True)
Gpath='gdrive/My Drive/'
training_data = np.load(Gpath+'training_data.npy', allow_pickle=True)

In [0]:
def showImg(data):
    for img in data:
        key = cv2.waitKey(0)
        cv2.imshow(str(img[1]),img[0])
        print(img[1])
        if(key == ord('q')): break

def imchange2(mask):
    mask = cv2.resize(mask, ( mask.shape[1], int(mask.shape[0]*2)))
    if mask.shape[0] >= mask.shape[1]:
        j = int(IMG_SIZE * mask.shape[1] / mask.shape[0])
        i = IMG_SIZE
    else:
        i = int(IMG_SIZE * mask.shape[0] / mask.shape[1])
        j = IMG_SIZE
    mask = cv2.resize(mask, (j, i))
    temp = np.zeros((IMG_SIZE, IMG_SIZE), np.uint8)
    yi = np.bincount(mask.reshape(-1))
    max_color = np.where(yi == max(yi[:-1]))[0][0]
    temp+=255
    if i == IMG_SIZE:
        jt = int((IMG_SIZE - j) / 2)
        temp[0:i, jt:jt + j] = mask
    else:
        it = int((IMG_SIZE - i) / 2)
        temp[it:it + i, 0:j] = mask
    return cv2.bitwise_not(temp)

def create_train_data(N):
  for img in tqdm(os.listdir(TRAIN_DIR+labelz[N])):
      path = os.path.join(TRAIN_DIR+labelz[N], img)
      img_data = cv2.imread(path,0)
      img_data = img_data.reshape((1,)+img_data.shape+(1,))
      i=0
      for batch in datagen.flow(img_data):
        if(i>4): break;
        i+=1
        batch = batch.astype(np.uint8)
        batch = batch.reshape(batch.shape[1], batch.shape[2])
        final_img = imchange2(batch) 
        training_data.append([np.array(final_img), str(N)])


In [0]:
# training_data = np.load('training_data.npy', allow_pickle=True)
print('Shuffling')
np.random.shuffle(training_data)
print('Done Shuffling!!!')
print(training_data.shape[0])
# showImg(training_data)


train = training_data[:-int(training_data.shape[0]*0.25)]
test = training_data[-int(training_data.shape[0]*0.25):]

X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train=to_categorical([int(i[1]) for i in train])
X_test = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test=to_categorical([int(i[1]) for i in test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Shuffling
Done Shuffling!!!
10600
(7950, 300, 300, 1) (7950, 14)
(2650, 300, 300, 1) (2650, 14)


In [0]:
model = []
model = Sequential()
model.add(Conv2D(96, (11, 11),strides=(4, 4), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))
model.add(Dropout(0.25))

model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(14, activation='softmax')) # 2 because we have cat and dog classes

model.summary()

W0825 10:53:08.103252 140011952609152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 10:53:08.147047 140011952609152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 10:53:08.153720 140011952609152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0825 10:53:08.189512 140011952609152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0825 10:53:08.190692 1400119526

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 96)        11712     
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 36, 36, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 256)       0         
__________

In [0]:
opt = Adam(lr=LR)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
model.fit(X_train, y_train, epochs=20, batch_size=25,
          validation_data=(X_test, y_test))

model.save("VISTA.model")

W0825 10:53:43.300433 140011952609152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0825 10:53:43.392275 140011952609152 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7950 samples, validate on 2650 samples
Epoch 1/20
7950/7950 [==============================] - 27s 3ms/step - loss: 2.6106 - acc: 0.1531 - val_loss: 2.4057 - val_acc: 0.1940
Epoch 2/20
7950/7950 [==============================] - 21s 3ms/step - loss: 2.4503 - acc: 0.1813 - val_loss: 2.4412 - val_acc: 0.1811
Epoch 3/20
7950/7950 [==============================] - 21s 3ms/step - loss: 2.4438 - acc: 0.1706 - val_loss: 3.3818 - val_acc: 0.1913
Epoch 4/20
7950/7950 [==============================] - 21s 3ms/step - loss: 2.3316 - acc: 0.2038 - val_loss: 2.4712 - val_acc: 0.1834
Epoch 5/20
7950/7950 [==============================] - 21s 3ms/step - loss: 2.2293 - acc: 0.2322 - val_loss: 2.1610 - val_acc: 0.2623
Epoch 6/20
7950/7950 [==============================] - 22s 3ms/step - loss: 2.1341 - acc: 0.2694 - val_loss: 1.8146 - val_acc: 0.3883
Epoch 7/20
7950/7950 [==============================] - 22s 3ms/step - loss: 1.9251 - acc: 0.3206 - val_loss: 1.6137 - val_acc: 0.4657
Epoch 8

In [0]:
opt = Adam(lr=LR)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=30,
          validation_data=(X_test, y_test))

model.save("VISTA.model")

Train on 7950 samples, validate on 2650 samples
Epoch 1/10
7950/7950 [==============================] - 23s 3ms/step - loss: 0.5016 - acc: 0.8530 - val_loss: 0.4613 - val_acc: 0.8668
Epoch 2/10
7950/7950 [==============================] - 19s 2ms/step - loss: 0.4197 - acc: 0.8717 - val_loss: 0.3296 - val_acc: 0.8989
Epoch 3/10
7950/7950 [==============================] - 20s 2ms/step - loss: 0.3958 - acc: 0.8809 - val_loss: 0.4316 - val_acc: 0.8951
Epoch 4/10
7950/7950 [==============================] - 20s 2ms/step - loss: 0.3533 - acc: 0.8908 - val_loss: 0.5627 - val_acc: 0.8257
Epoch 5/10
7950/7950 [==============================] - 20s 2ms/step - loss: 0.3384 - acc: 0.8991 - val_loss: 3.7463 - val_acc: 0.6019
Epoch 6/10
7950/7950 [==============================] - 20s 2ms/step - loss: 0.3690 - acc: 0.8940 - val_loss: 0.5320 - val_acc: 0.8509
Epoch 7/10
7950/7950 [==============================] - 20s 3ms/step - loss: 0.3321 - acc: 0.9001 - val_loss: 0.4083 - val_acc: 0.8943
Epoch 8

In [0]:
# earlystop = EarlyStopping(patience=10,verbose=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
patience=3 ,
verbose=1, 
factor=0.5, 
min_lr=0.00001)
# model = load_model('VISTAfinal.model')
save_model = ModelCheckpoint('bestModel.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
callbacks = [save_model, learning_rate_reduction]
opt = Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])

datagen = ImageDataGenerator(
        rotation_range=4,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
#         brightness_range = [0.2,1],
        fill_mode='nearest')
model.fit_generator(datagen.flow(X_train, y_train, batch_size=100),
                    steps_per_epoch=len(X_train)/100,epochs=10, callbacks = callbacks,
                    validation_data=(X_test, y_test))

Epoch 1/10
80/79 [==============================] - 48s 602ms/step - loss: 0.2035 - acc: 0.9446 - val_loss: 0.1227 - val_acc: 0.9660

Epoch 00001: val_loss improved from inf to 0.12267, saving model to bestModel.model
Epoch 2/10
80/79 [==============================] - 38s 469ms/step - loss: 0.2117 - acc: 0.9418 - val_loss: 0.1149 - val_acc: 0.9638

Epoch 00002: val_loss improved from 0.12267 to 0.11487, saving model to bestModel.model
Epoch 3/10
80/79 [==============================] - 40s 505ms/step - loss: 0.1978 - acc: 0.9449 - val_loss: 0.3157 - val_acc: 0.9026

Epoch 00003: val_loss did not improve from 0.11487
Epoch 4/10
80/79 [==============================] - 40s 496ms/step - loss: 0.2005 - acc: 0.9450 - val_loss: 0.2254 - val_acc: 0.9442

Epoch 00004: val_loss did not improve from 0.11487

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 5/10
80/79 [==============================] - 38s 474ms/step - loss: 0.1866 - acc: 0.9486 - val_loss: 0

In [0]:
Gpath='gdrive/My Drive/'
test_data = np.load(Gpath+'eee', allow_pickle=True)

In [0]:
! cp VISTA.model gdrive/My\ Drive/

In [0]:
model.save('VISTAfinal.model')

In [0]:
model = load_model('bestModel.model')


NameError: ignored

In [0]:
!cp VISTAfinal.model gdrive/My\ Drive/VISTAfinal.model
!cp bestModel.model gdrive/My\ Drive/bestModel.model

In [0]:
model = load_model('gdrive/My Drive/VISTAfinal.model')

In [0]:
X_test = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
name = np.array([i[1] for i in test_data])
X_test = X_test.astype('float32')
X_test = X_test/255

print(X_test.shape)


with open('result12.csv', 'a') as csvFile:
	writer = csv.writer(csvFile)
	for i in range(len(X_test)):
		class_prediction = model.predict_classes(X_test[i].reshape(1, IMG_SIZE, IMG_SIZE, 1), verbose = False)
		row = [name[i], class_prediction[0]+1]
		writer.writerow(row)

In [0]:
! cp result12.csv gdrive/My\ Drive/